In [2]:
import pandas as pd
import numpy as np


# Import the data and assign description column to process

In [3]:
csv = 'PI_NEWandOLD.csv'
issues_df = pd.read_csv(csv, encoding='latin1',low_memory=False)

In [ ]:
issues_df.head()

In [4]:
issues_df['description'] = issues_df['description'].fillna('').astype(str)
issues_df['title'] = issues_df['title'].fillna('').astype(str)

In [5]:
def description(row):
    if len(row['description']) > len(row['title']):
        return row['description2']
    if len(row['description']) < len(row['title']):
        return row['title']

In [6]:
issues_df['description2']=issues_df.apply (lambda row: description(row), axis =1)

In [7]:
description_text = issues_df[["description"]]
description_text['index']=description_text.index
documents = description_text

C:\Users\mrabier\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
print(len(documents))
print(documents[:30])

# Data Pre-processing which includes tokenization, removal of stop words and words with fewer than 3 words, lemmatization, stemming

In [8]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
np.random.seed(2018)
import nltk
nltk.download('wordnet')
from gensim.test.utils import datapath
from gensim import corpora, models, similarities

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mrabier\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [9]:
stemmer = PorterStemmer()

In [11]:
#Write a function to perform lemmatize and stem preprocessing steps on the data set

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [12]:
#Select a document to preview after preprocessing

doc_sample = documents[documents['index'] == 4310].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['Remove', 'paint', 'from', 'rubber', 'base', 'left', 'of', 'room', '606']


 tokenized and lemmatized document: 
['remov', 'paint', 'rubber', 'base', 'leav', 'room']


In [29]:
# Pre-process all of the description cells

processed_docs = documents['description'].dropna('').astype(str).map(preprocess)
processed_docs[:30]

0                             [remov, nail, dock, angl]
1     [grind, chamfer, exterior, corner, tilt, panel...
2                  [instal, miss, switch, data, outlet]
3                 [instal, cover, plate, light, switch]
4                                        [instal, base]
5                         [repaint, north, south, wall]
6                     [complet, strip, traffic, signag]
7                                        [clean, floor]
8                    [touch, yellow, paint, rail, door]
9                                                [door]
10              [nail, hole, door, window, offic, mezz]
11                                  [paint, east, wall]
12                                        [paint, door]
13    [verifi, base, heater, women, clayco, respons,...
14    [patch, drive, ramp, level, length, chamfer, c...
15                            [clean, haze, tile, wall]
16     [remov, stick, insul, drywal, deck, draw, paint]
17                [replac, thermax, electr, room

# Bag of Words on the Data Set

In [14]:
dictionary = gensim.corpora.Dictionary(processed_docs)


In [ ]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 30:
        break

In [17]:
# filter out tokens that appear in less than 15 documents, more than .5 documents and the most 100000 most frequent tokens
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=10000)

In [ ]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 30:
        break

In [16]:
#For each cell, we create a dictionary reporting how many words and how many times those words appear
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(3, 1), (23, 1), (35, 1), (64, 1), (164, 1), (401, 1)]

In [18]:
bow_doc_4310 = bow_corpus[4310]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
bow_doc_4310[i][1]))

Word 3 ("remov") appears 1 time.
Word 23 ("base") appears 1 time.
Word 35 ("paint") appears 1 time.
Word 64 ("show") appears 1 time.
Word 164 ("safe") appears 1 time.
Word 401 ("sealant") appears 1 time.


In [19]:
# TF-IDF

from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.575943665995411),
 (1, 0.5680507148758992),
 (2, 0.5122679097636066),
 (3, 0.28842480398740267)]


# Run LDA using Bag of Words

In [ ]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=20, id2word=dictionary, passes=4, workers=2)

In [ ]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

In [ ]:
# Save model to disk.
lda_model.save('lda.model')

In [20]:
# Load a potentially pretrained model from disk.
model =  models.LdaModel.load('lda.model')

# Run LDA using TF-IDF

In [ ]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=20, id2word=dictionary, passes=4, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

In [ ]:
# Save model to disk.
#lda_model_tfidf.save('lda tfidf.model')

In [ ]:
#Performance evaluation 
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 5)))

In [21]:
model =  models.LdaModel.load('lda tfidf.model')

In [ ]:
# I can print out the documents and which is the most probable topics for each doc.
corpus_lda = model[corpus_tfidf]

for l,t in zip(corpus_lda,corpus_tfidf):
  print(l,"#",t)
print


In [ ]:
type(processed_docs)

In [ ]:
get_document_topics = [model.get_document_topics(item) for item in bow_corpus]

In [ ]:
get_document_topics

In [32]:
def format_topics_sentences(ldamodel=None, corpus=corpus_tfidf, texts=processed_docs):
    # Init output
    sent_topics_df = pd.DataFrame()
    # Get main topic in each document
    try:
        for i, row_list in enumerate(model[corpus]):
            row = row_list[0] if model.per_word_topics else row_list            
            print(row)
            row = sorted(row, key=lambda x: (x[1]), reverse=True)
            # Get the Dominant topic, Perc Contribution and Keywords for each document
            for j, (topic_num, prop_topic) in enumerate(row):
                if j == 0:  # => dominant topic
                    wp = model.show_topic(topic_num)
                    topic_keywords = ", ".join([word for word, prop in wp])
                    sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
                else:
                    break
        sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']
    except:
        pass
    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)




In [33]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=model, corpus=corpus_tfidf, texts=processed_docs)

[(0, 0.016980527), (1, 0.016980527), (2, 0.016980527), (3, 0.016980527), (4, 0.4209915), (5, 0.016980527), (6, 0.016980527), (7, 0.016980527), (8, 0.016980527), (9, 0.016980527), (10, 0.016980527), (11, 0.016980527), (12, 0.016980527), (13, 0.016980527), (14, 0.016980527), (15, 0.016980527), (16, 0.27335903), (17, 0.016980527), (18, 0.016980529), (19, 0.016980527)]
[(0, 0.012307091), (1, 0.012307089), (2, 0.012307089), (3, 0.012307089), (4, 0.5913769), (5, 0.012307089), (6, 0.012307089), (7, 0.012307089), (8, 0.012307089), (9, 0.012307091), (10, 0.012307089), (11, 0.012307089), (12, 0.012307089), (13, 0.012307089), (14, 0.012307089), (15, 0.18709548), (16, 0.012307089), (17, 0.012307089), (18, 0.012307089), (19, 0.012307089)]
[(0, 0.015954899), (1, 0.6968569), (2, 0.015954899), (3, 0.015954899), (4, 0.015954899), (5, 0.015954899), (6, 0.015954899), (7, 0.015954899), (8, 0.015954899), (9, 0.015954899), (10, 0.015954899), (11, 0.015954899), (12, 0.015954899), (13, 0.015954899), (14, 0.01

[(0, 0.014783634), (1, 0.014783634), (2, 0.014783634), (3, 0.014783634), (4, 0.23098998), (5, 0.014783634), (6, 0.093105644), (7, 0.014783634), (8, 0.16008322), (9, 0.279283), (10, 0.014783634), (11, 0.014783634), (12, 0.014783634), (13, 0.014783634), (14, 0.014783634), (15, 0.014783634), (16, 0.014783634), (17, 0.014783634), (18, 0.014783635), (19, 0.014783634)]
[(0, 0.017095504), (1, 0.017095506), (2, 0.017095504), (3, 0.017095506), (4, 0.017095506), (5, 0.017095504), (6, 0.017095504), (7, 0.017095506), (8, 0.017095504), (9, 0.017095508), (10, 0.017095504), (11, 0.017095504), (12, 0.017095506), (13, 0.017095504), (14, 0.6751854), (15, 0.017095504), (16, 0.017095504), (17, 0.017095504), (18, 0.017095506), (19, 0.017095504)]
[(0, 0.016873125), (1, 0.016873127), (2, 0.016873129), (3, 0.016873125), (4, 0.016873125), (5, 0.016873125), (6, 0.016873127), (7, 0.016873125), (8, 0.6794106), (9, 0.016873125), (10, 0.016873125), (11, 0.016873125), (12, 0.016873125), (13, 0.016873127), (14, 0.016

[(0, 0.021183774), (1, 0.021183776), (2, 0.021183774), (3, 0.021183774), (4, 0.021183774), (5, 0.021183774), (6, 0.021183776), (7, 0.021183774), (8, 0.021183774), (9, 0.021183774), (10, 0.021183774), (11, 0.021183774), (12, 0.021183774), (13, 0.021183774), (14, 0.021183774), (15, 0.021183776), (16, 0.021183774), (17, 0.5975084), (18, 0.021183774), (19, 0.021183774)]
[(0, 0.021124365), (1, 0.021124365), (2, 0.021124365), (3, 0.021124365), (4, 0.021124365), (5, 0.021124365), (6, 0.021124367), (7, 0.021124365), (8, 0.5986371), (9, 0.021124365), (10, 0.021124365), (11, 0.021124365), (12, 0.021124365), (13, 0.021124365), (14, 0.021124365), (15, 0.021124367), (16, 0.021124365), (17, 0.021124365), (18, 0.021124365), (19, 0.021124365)]
[(0, 0.014692514), (1, 0.014692514), (2, 0.014692514), (3, 0.014692514), (4, 0.3408867), (5, 0.014692514), (6, 0.014692514), (7, 0.014692515), (8, 0.014692514), (9, 0.014692514), (10, 0.014692514), (11, 0.014692514), (12, 0.2593501), (13, 0.014692514), (14, 0.01

[(0, 0.016103037), (1, 0.016103037), (2, 0.016103037), (3, 0.016103037), (4, 0.016103037), (5, 0.23534523), (6, 0.47480008), (7, 0.016103037), (8, 0.016103037), (9, 0.016103037), (10, 0.016103037), (11, 0.016103037), (12, 0.016103037), (13, 0.016103037), (14, 0.016103037), (15, 0.016103037), (16, 0.016103037), (17, 0.016103037), (18, 0.016103037), (19, 0.016103037)]
[(0, 0.018383086), (1, 0.018383086), (2, 0.018383086), (3, 0.018383086), (4, 0.018383086), (5, 0.018383086), (6, 0.018383086), (7, 0.018383086), (8, 0.018383086), (9, 0.22183691), (10, 0.018383086), (11, 0.018383086), (12, 0.20563379), (13, 0.018383086), (14, 0.018383086), (15, 0.018383086), (16, 0.018383086), (17, 0.26001686), (18, 0.018383086), (19, 0.018383086)]
[(0, 0.019277636), (1, 0.17107323), (2, 0.48192927), (3, 0.019277636), (4, 0.019277636), (5, 0.019277636), (6, 0.019277636), (7, 0.019277636), (8, 0.019277636), (9, 0.019277636), (10, 0.019277636), (11, 0.019277636), (12, 0.019277636), (13, 0.019277636), (14, 0.0

[(0, 0.66597646), (1, 0.017580181), (2, 0.017580185), (3, 0.017580183), (4, 0.017580181), (5, 0.017580181), (6, 0.017580185), (7, 0.017580187), (8, 0.017580185), (9, 0.017580183), (10, 0.017580181), (11, 0.017580181), (12, 0.017580181), (13, 0.017580183), (14, 0.017580183), (15, 0.017580181), (16, 0.017580183), (17, 0.017580183), (18, 0.017580181), (19, 0.017580181)]
[(0, 0.018570507), (1, 0.018570509), (2, 0.018570507), (3, 0.018570507), (4, 0.018570507), (5, 0.018570507), (6, 0.018570507), (7, 0.018570507), (8, 0.018570507), (9, 0.018570507), (10, 0.018570507), (11, 0.018570507), (12, 0.3165926), (13, 0.018570507), (14, 0.018570507), (15, 0.018570507), (16, 0.018570507), (17, 0.12446397), (18, 0.24324484), (19, 0.018570507)]
[(0, 0.018499563), (1, 0.018499563), (2, 0.018499563), (3, 0.018499563), (4, 0.018499563), (5, 0.018499563), (6, 0.018499563), (7, 0.018499563), (8, 0.018499563), (9, 0.018499563), (10, 0.018499563), (11, 0.018499563), (12, 0.018499563), (13, 0.018499563), (14, 0

...

[(0, 0.016120462), (1, 0.24617423), (2, 0.016120462), (3, 0.016120462), (4, 0.016120462), (5, 0.016120462), (6, 0.016120462), (7, 0.016120462), (8, 0.20775518), (9, 0.016120462), (10, 0.016120462), (11, 0.016120462), (12, 0.016120462), (13, 0.016120462), (14, 0.016120462), (15, 0.016120464), (16, 0.27202275), (17, 0.016120462), (18, 0.016120462), (19, 0.016120462)]
[(0, 0.019110685), (1, 0.019110685), (2, 0.019110685), (3, 0.3235878), (4, 0.019110685), (5, 0.019110685), (6, 0.019110685), (7, 0.019110685), (8, 0.019110685), (9, 0.019110685), (10, 0.3324199), (11, 0.019110685), (12, 0.019110685), (13, 0.019110685), (14, 0.019110685), (15, 0.019110685), (16, 0.019110685), (17, 0.019110685), (18, 0.019110685), (19, 0.019110685)]
[(0, 0.0629186), (1, 0.12905674), (2, 0.010544181), (3, 0.010544181), (4, 0.0586078), (5, 0.010544181), (6, 0.010544181), (7, 0.072205), (8, 0.010544189), (9, 0.098320946), (10, 0.2170812), (11, 0.010544181), (12, 0.05177568), (13, 0.010544181), (14, 0.0797213), (1

[(0, 0.013948776), (1, 0.013948776), (2, 0.013948776), (3, 0.013948776), (4, 0.092652805), (5, 0.117232695), (6, 0.096374705), (7, 0.013948776), (8, 0.1496281), (9, 0.013948776), (10, 0.115459695), (11, 0.013948776), (12, 0.013948776), (13, 0.013948776), (14, 0.14326464), (15, 0.013948776), (16, 0.013948776), (17, 0.013948776), (18, 0.10405324), (19, 0.013948776)]
[(0, 0.10496335), (1, 0.01192848), (2, 0.01192848), (3, 0.01192848), (4, 0.011928481), (5, 0.01192848), (6, 0.01192848), (7, 0.01192848), (8, 0.121043295), (9, 0.3369107), (10, 0.01192848), (11, 0.01192848), (12, 0.011928481), (13, 0.01192848), (14, 0.13669512), (15, 0.12146035), (16, 0.011928487), (17, 0.01192848), (18, 0.011928482), (19, 0.01192848)]
[(0, 0.015143024), (1, 0.015143024), (2, 0.21662872), (3, 0.015143024), (4, 0.015143025), (5, 0.015143024), (6, 0.015143024), (7, 0.015143024), (8, 0.015143025), (9, 0.17006937), (10, 0.015143024), (11, 0.015143024), (12, 0.10056071), (13, 0.015143024), (14, 0.015143025), (15, 

[(0, 0.040215824), (1, 0.040215824), (2, 0.04021582), (3, 0.04021582), (4, 0.04021582), (5, 0.04021582), (6, 0.040215824), (7, 0.040215824), (8, 0.04021584), (9, 0.040215824), (10, 0.04021582), (11, 0.040215824), (12, 0.04021582), (13, 0.04021582), (14, 0.040215824), (15, 0.040215824), (16, 0.23589933), (17, 0.04021582), (18, 0.040215824), (19, 0.040215824)]
[(0, 0.018629368), (1, 0.018629367), (2, 0.018629367), (3, 0.018629367), (4, 0.018629367), (5, 0.018629367), (6, 0.018629368), (7, 0.018629367), (8, 0.018629368), (9, 0.018629367), (10, 0.018629367), (11, 0.018629367), (12, 0.018629368), (13, 0.018629367), (14, 0.64604205), (15, 0.018629367), (16, 0.018629367), (17, 0.01862937), (18, 0.018629367), (19, 0.018629367)]
[(0, 0.018461322), (1, 0.018461322), (2, 0.018461322), (3, 0.27500126), (4, 0.018461322), (5, 0.018461322), (6, 0.19024678), (7, 0.018461322), (8, 0.018461322), (9, 0.018461322), (10, 0.018461322), (11, 0.018461322), (12, 0.018461322), (13, 0.018461322), (14, 0.22090948

[(0, 0.2703544), (1, 0.017126933), (2, 0.017126933), (3, 0.017126933), (4, 0.017126933), (5, 0.017126933), (6, 0.017126933), (7, 0.20799111), (8, 0.017126933), (9, 0.017126933), (10, 0.017126933), (11, 0.017126933), (12, 0.017126933), (13, 0.017126933), (14, 0.017126933), (15, 0.017126933), (16, 0.017126933), (17, 0.23049664), (18, 0.017126933), (19, 0.017126933)]
[(0, 0.019149631), (1, 0.01914963), (2, 0.01914963), (3, 0.019149635), (4, 0.019149633), (5, 0.019149631), (6, 0.01914963), (7, 0.636157), (8, 0.019149633), (9, 0.01914963), (10, 0.01914963), (11, 0.01914963), (12, 0.019149631), (13, 0.01914963), (14, 0.019149631), (15, 0.01914963), (16, 0.01914969), (17, 0.01914963), (18, 0.01914963), (19, 0.01914963)]
[(0, 0.0183822), (1, 0.0183822), (2, 0.0183822), (3, 0.0183822), (4, 0.0183822), (5, 0.0183822), (6, 0.0183822), (7, 0.65073824), (8, 0.0183822), (9, 0.0183822), (10, 0.0183822), (11, 0.0183822), (12, 0.0183822), (13, 0.0183822), (14, 0.0183822), (15, 0.0183822), (16, 0.018382

[(0, 0.024866464), (1, 0.024866464), (2, 0.5275372), (3, 0.024866464), (4, 0.024866464), (5, 0.024866464), (6, 0.024866464), (7, 0.024866464), (8, 0.024866464), (9, 0.024866464), (10, 0.024866464), (11, 0.024866464), (12, 0.024866464), (13, 0.024866464), (14, 0.024866464), (15, 0.024866464), (16, 0.024866464), (17, 0.024866464), (18, 0.024866464), (19, 0.024866464)]
[(0, 0.022044946), (1, 0.022044946), (2, 0.022044951), (3, 0.022044947), (4, 0.022044946), (5, 0.022044947), (6, 0.022044955), (7, 0.022044955), (8, 0.022044946), (9, 0.022044947), (10, 0.581146), (11, 0.022044947), (12, 0.022044946), (13, 0.022044947), (14, 0.022044947), (15, 0.022044946), (16, 0.022044947), (17, 0.022044946), (18, 0.022044946), (19, 0.022044946)]
[(0, 0.020280333), (1, 0.020280333), (2, 0.020280333), (3, 0.020280335), (4, 0.020280333), (5, 0.020280333), (6, 0.020280333), (7, 0.020280333), (8, 0.15112695), (9, 0.020280333), (10, 0.020280333), (11, 0.020280335), (12, 0.11472997), (13, 0.020280333), (14, 0.0

In [34]:
df_topic_sents_keywords

,0,1,2,description
0,4.0,0.4210,"column, attach, photo, chip, concret, dock, un...","[remov, nail, dock, angl]"
1,4.0,0.5914,"column, attach, photo, chip, concret, dock, un...","[grind, chamfer, exterior, corner, tilt, panel..."
2,1.0,0.6969,"cover, outlet, plate, switch, receptacl, insta...","[instal, miss, switch, data, outlet]"
3,1.0,0.6992,"cover, outlet, plate, switch, receptacl, insta...","[instal, cover, plate, light, switch]"
4,15.0,0.5972,"base, instal, cabinet, complet, floor, wall, s...","[instal, base]"
...,...,...,...,...
159280,NaN,NaN,NaN,"[own, chewi, need, remov]"
159281,NaN,NaN,NaN,[]
159282,NaN,NaN,NaN,"[need, creat, sure, pick, area, allig, asphalt..."
159283,NaN,NaN,NaN,"[need, come, plan, repair, slab, probabl, need..."


In [35]:
# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
df_dominant_topic.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,4.0,0.4210,"column, attach, photo, chip, concret, dock, un...","[remov, nail, dock, angl]"
1,1,4.0,0.5914,"column, attach, photo, chip, concret, dock, un...","[grind, chamfer, exterior, corner, tilt, panel..."
2,2,1.0,0.6969,"cover, outlet, plate, switch, receptacl, insta...","[instal, miss, switch, data, outlet]"
3,3,1.0,0.6992,"cover, outlet, plate, switch, receptacl, insta...","[instal, cover, plate, light, switch]"
4,4,15.0,0.5972,"base, instal, cabinet, complet, floor, wall, s...","[instal, base]"
5,5,3.0,0.6768,"patch, hole, wall, paint, clean, room, repaint...","[repaint, north, south, wall]"
6,6,8.0,0.3109,"protect, smoke, detector, remov, window, descr...","[complet, strip, traffic, signag]"
7,7,15.0,0.6063,"base, instal, cabinet, complet, floor, wall, s...","[clean, floor]"
8,8,7.0,0.2806,"touch, paint, frame, door, wall, finish, latch...","[touch, yellow, paint, rail, door]"
9,9,7.0,0.5250,"touch, paint, frame, door, wall, finish, latch...",[door]


In [39]:
df_dominant_topic.to_excel('Dominant Topics.xlsx', index = False, header = True)
